# 구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 경로 이동

In [2]:
%cd /content/drive/MyDrive/rhee/code/halibut

/content/drive/MyDrive/rhee/code/halibut


# requirements.txt

In [3]:
!pip install ultralytics
!pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 torchaudio==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install numpy
!pip install pandas
!pip install Pillow
!pip install pyYAML
!pip install requests
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# convert coco to yolo

In [4]:
import os
import json

In [5]:
with open("datasets/sample_dataset/annotations/train_annotations.coco.json", "r") as train_json:
    train_dataset = json.load(train_json)
    train_json.close()

with open("datasets/sample_dataset/annotations/valid_annotations.coco.json", "r") as valid_json:
    valid_dataset = json.load(valid_json)
    valid_json.close()

with open("datasets/sample_dataset/annotations/test_annotations.coco.json", "r") as test_json:
    test_dataset = json.load(test_json)
    test_json.close()

In [6]:
def json2yolo(dataset, save_path):
    # 폴더 생성 이미 폴더가 있으면 실행 안함.
    os.makedirs(save_path, exist_ok=True)
    # 클래스 txt 파일 생성
    # json 파일 내 클래스를 리스트에 저장해서 txt 파일로 쓰기.
    categories = [category["name"] for category in dataset["categories"]]
    save_categories_name = os.path.join(save_path, "classes.txt")
    lines = ""
    for category in categories:
        lines += f"{category}\n"
    # 클래스 txt 파일 쓰기
    with open(save_categories_name, "w") as f:
        f.writelines(lines)
        f.close()
    # dataset image (파일 이름, 너비, 높이) annotations xyxy 좌표 추출 및 txt로 변환 
    for img, anno in zip(dataset["images"], dataset["annotations"]):
        lines = ""
        file_name = img["file_name"] # 파일 이름 추출
        save_name = os.path.join(save_path, file_name).replace(".jpg", ".txt") # 저장할 txt 파일 이름
        height = img["height"] # 아미지 높이
        width = img["width"] # 이미지 너비
        category = anno["category_id"] # 클래스 인덱스
        bbox = anno["bbox"] # xyxy 좌표
        x1, y1, x2, y2 = bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3] # xyxy 좌표를 낱개로 초기화
        
        # yolo txt 형식으로 변환
        x = (x1+x2)/2/width
        y = (y1+y2)/2/height
        w = (x2-x1)/width
        h = (y2-y1)/height
        lines += f"{category} {round(x,6)} {round(y,6)} {round(w,6)} {round(h,6)}\n"
        # yolo txt 파일 쓰기.
        with open(save_name, "w") as f:
            f.writelines(lines)
            f.close()

In [7]:
json2yolo(train_dataset, "datasets/sample_dataset/train/labels") # train labels 파일 생성
json2yolo(valid_dataset, "datasets/sample_dataset/valid/labels") # valid labels 파일 생성
json2yolo(test_dataset, "datasets/sample_dataset/test/labels") # test labels 파일 생성

# train

In [8]:
from ultralytics import YOLO # yolov8 패키지 import

In [13]:
# upadte settings.yaml // yolov8이 현재 가지고 있는 이슈 // yolov8 저자도 yolov8 이렇게 settings 파일 내에서 고쳐야한다고 함.
# yolov8을 맨처음 설치하고 수행시 상관 없음/ 그러나 경로가 바뀌어지면 해당 파일을 직접 변경해야함.
import yaml

with open('/root/.config/Ultralytics/settings.yaml') as f:
    settings = yaml.load(f, Loader=yaml.FullLoader)
    f.close()

settings["datasets_dir"] = "/content/drive/MyDrive/rhee/code/halibut"

with open('/root/.config/Ultralytics/settings.yaml', 'w') as f:
    print(settings)
    yaml.dump(settings, f)
    f.close()

{'api_key': '', 'datasets_dir': '/content/drive/MyDrive/rhee/code/halibut', 'runs_dir': 'runs', 'settings_version': '0.0.3', 'sync': True, 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb', 'weights_dir': 'weights'}


In [10]:
# 모델 아키텍처 및 pretrained 모델 로드
model = YOLO('yolov8s.yaml').load('pretrained/yolov8s.pt')

# 학습 코드 // custom.yaml 파일에서 hyper-parameter setting을 조정 epoch , learning rate , .. 등
model.train(cfg="cfg/custom.yaml")


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

# predict

In [11]:
from ultralytics import YOLO
import numpy as np
from matplotlib import pyplot as plt
import glob
import cv2

# 학습한 모델 불러오기.
# model = YOLO(model="runs/sample/weights/best.pt", task="detect")
model = YOLO(model="pretrained/yolov8s.pt", task="detect")

In [12]:
%matplotlib inline # plt.show() 로 보기위해서 해당 코드가 필요함.
# test 데이터셋의 이미지들을 glob모듈을 활용해서 로드
for im_path in glob.glob("../datasets/sample_dataset/test/images/*.jpg"):
    img = cv2.imread(im_path, cv2.IMREAD_COLOR) # numpy array로 이미지 로드
    results = model.predict(source=img, verbose=False, iou=0.75, device=0)
    """
    예측 코드, 
    verbose=False -> yolov8 output을 무시,
    iou=0.75 -> predict 교집합 ground_truth / predict 합집합 ground_truth 를 0.75 수준으로 진행
    device=0 -> 0번째 gpu를 사용하겠다는 것 없으면 cpu로 진행
    """
    for result in results:
        score = result.boxes.conf.cpu().numpy()  # 객체 당 score tensor 리스트
        classes = result.boxes.cls.cpu().numpy()  # 객체 당 class 리스트
        xyxy = result.boxes.xyxy.cpu().numpy()  # 객체 당 xyxy 리스트

        for i in range(len(classes)):
            if score[i] >= 0.7: # confidence score가 0.7 이상인 것만 detection
                x1, y1, x2, y2 = [int(x) for x in xyxy[i]] # x1, y1, x2, y2 좌표 추출
                img = cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 3) # input bounding box
                img = cv2.putText(img, str(classes[i]), (x1, y1+30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3) # input text
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()